In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/dhoogla/unswnb15?resource=download&select=UNSW_NB15_training-set.parquet')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: pramodrathnayake
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/dhoogla/unswnb15


100%|██████████| 11.7M/11.7M [00:00<00:00, 1.47GB/s]

In [20]:
import pandas as pd
df = pd.read_parquet('/content/unswnb15/UNSW_NB15_training-set.parquet')

In [21]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Embedding,Input,Flatten,Dense
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()

import numpy as np

In [22]:
cat_cols = ['service', 'state']

df = pd.get_dummies(df, columns=cat_cols, dtype=int)

In [23]:
num_cols = df.select_dtypes(include=['int64', 'float64','float32','int8','int16','int32']).columns
num_cols = num_cols.drop('proto', errors='ignore')
num_cols = num_cols.drop('label', errors='ignore')
df[num_cols] = scaler.fit_transform(df[num_cols])

In [24]:
le = LabelEncoder()
X_proto = le.fit_transform(df['proto'])
X_num = df[num_cols].values
y_data = df['label'].values

proto_in = Input(shape=(1,),name = 'proto_in')
proto_emb = Embedding(input_dim=133, output_dim=8,)(proto_in)

proto_vec = Flatten()(proto_emb)

In [25]:
num_in = Input(shape=(X_num.shape[1],), name='other_input')


x = layers.Concatenate()([proto_vec,num_in])

In [26]:
from sklearn.model_selection import train_test_split

X_proto_train, X_proto_val, X_num_train, X_num_val, y_train, y_val = train_test_split(
    X_proto, X_num, y_data,
    test_size=0.2,
    random_state=42,
    stratify=y_data
)

In [27]:
X_proto_train.shape

(140272,)

In [28]:
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(16, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs=[proto_in, num_in], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')] )
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ proto_in            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 8)      │      1,064 │ proto_in[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 8)         │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ other_input         │ (None, 53)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 61)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ other_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │      3,968 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 64)        │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      2,080 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 32)        │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      2,112 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 64)        │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 32)        │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │        528 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         17 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,849 (46.29 KB)

 Trainable params: 11,849 (46.29 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
X_num_train.shape, X_proto_train.shape

((140272, 53), (140272,))

In [30]:
history = model.fit(
    {'proto_in': X_proto_train, 'other_input': X_num_train},
    y=y_train,
    epochs=20,
    batch_size=64,
    verbose=1,
    validation_data=(
        {'proto_in': X_proto_val, 'other_input': X_num_val},
        y_val
    )
)

Epoch 1/20
2192/2192 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8927 - auc: 0.9381 - loss: 0.2611 - precision: 0.8914 - recall: 0.9579 - val_accuracy: 0.9333 - val_auc: 0.9769 - val_loss: 0.1555 - val_precision: 0.9136 - val_recall: 0.9962
Epoch 2/20
2192/2192 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9292 - auc: 0.9759 - loss: 0.1621 - precision: 0.9109 - recall: 0.9933 - val_accuracy: 0.9329 - val_auc: 0.9806 - val_loss: 0.1465 - val_precision: 0.9117 - val_recall: 0.9981
Epoch 3/20
2192/2192 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9309 - auc: 0.9790 - loss: 0.1517 - precision: 0.9143 - recall: 0.9913 - val_accuracy: 0.9338 - val_auc: 0.9827 - val_loss: 0.1403 - val_precision: 0.9129 - val_recall: 0.9979
Epoch 4/20
2192/2192 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9317 - auc: 0.9808 - loss: 0.1461 - precision: 0.9141 - recall: 0.9928 - val_accuracy: 0.9347 - val_auc: 0.9829 - val_loss: 0.1393 - val_precision: 0.9144 - val_recall: 0.9975
Epoch 5/20
2192/2192 ━